In [29]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [30]:
rent_obj = []
all_property = []

for page in range(25,29):
    link = 'https://www.apartments.com/houses/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     page 1 - 2
#     results = soup.find_all('article', class_='premiumPlus placard')
#     page 2-19
#     results = soup.find_all('article', class_='premium placard')
#     page 19-28
    results = soup.find_all('article', class_='prosumer placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Houses",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

[{'City_Rent': '10231 Lotus Ct', 'Type': 'Houses', 'State_Rent': 'Stanton', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '4 Beds', 'price': '$3,000'}], 'Full_Address': 'Stanton, CA 90680', 'Rent_link': 'https://www.apartments.com/10231-lotus-ct-stanton-ca/3s6v3qf/', 'Bath_Info': '3 BAs', 'Square_Feet_Data': '1,616 Sq Ft', 'Square_Feet': '1,616', 'Lease_length': '', 'Availability_check': 'Available Now', '4 Beds': '$3,000'}, {'City_Rent': '16960 Mockingbird Canyon Rd', 'Type': 'Houses', 'State_Rent': 'Riverside', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '3 Beds', 'price': '$2,500'}], 'Full_Address': 'Riverside, CA 92504', 'Rent_link': 'https://www.apartments.com/16960-mockingbird-canyon-rd-riverside-ca/2bly6gb/', 'Bath_Info': '3 BAs', 'Square_Feet_Data': '2,452 Sq Ft', 'Square_Feet': '2,452', 'Lease_length': '', 'Availability_check': 'Available Now', '3 Beds': '$2,500'}, {'City_Rent': '22831 Montanya Pl', 'Type': 'Houses', 'State_Rent': 'Murrieta', 'Zip_code': 'CA', 'B

In [31]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,4 Beds,3 Beds,4+ Beds,2 Beds,1 Bed,Studio
0,10231 Lotus Ct,Houses,Stanton,CA,"[{'bedroom_type': '4 Beds', 'price': '$3,000'}]","Stanton, CA 90680",https://www.apartments.com/10231-lotus-ct-stan...,3 BAs,"1,616 Sq Ft","1,616",,Available Now,"$3,000",NaN,NaN,NaN,NaN,NaN
1,16960 Mockingbird Canyon Rd,Houses,Riverside,CA,"[{'bedroom_type': '3 Beds', 'price': '$2,500'}]","Riverside, CA 92504",https://www.apartments.com/16960-mockingbird-c...,3 BAs,"2,452 Sq Ft","2,452",,Available Now,NaN,"$2,500",NaN,NaN,NaN,NaN
2,22831 Montanya Pl,Houses,Murrieta,CA,"[{'bedroom_type': '4+ Beds', 'price': '$2,995'}]","Murrieta, CA 92562",https://www.apartments.com/22831-montanya-pl-m...,4 BAs,"3,646 Sq Ft","3,646",,Available Now,NaN,NaN,"$2,995",NaN,NaN,NaN
3,28051 Calle Casera,Houses,Menifee,CA,"[{'bedroom_type': '2 Beds', 'price': '$1,450'}]","Menifee, CA 92585",https://www.apartments.com/28051-calle-casera-...,2 BAs,"1,101 Sq Ft","1,101",,Available Now,NaN,NaN,NaN,"$1,450",NaN,NaN
4,2532 Northshore Ln,Houses,Westlake Village,CA,"[{'bedroom_type': '3 Beds', 'price': '$6,500'}]","Westlake Village, CA 91361",https://www.apartments.com/2532-northshore-ln-...,2 BAs,,,,Available Now,NaN,"$6,500",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Templeton,Houses,CA,93465,"[{'bedroom_type': '4 Beds', 'price': '$2,575'}]","Templeton, CA 93465",https://www.apartments.com/1150-megan-ct-templ...,2½ BAs,"2,450 Sq Ft","2,450",36 Month Lease,Jul 15,"$2,575",NaN,NaN,NaN,NaN,NaN
96,Stockton,Houses,CA,95207,"[{'bedroom_type': '2 Beds', 'price': '$1,605'}]","Stockton, CA 95207",https://www.apartments.com/2656-michaelangelo-...,2 BAs,"1,195 Sq Ft","1,195",1 Month Lease,Available Now,NaN,NaN,NaN,"$1,605",NaN,NaN
97,Pleasanton,Houses,CA,94566,"[{'bedroom_type': '3 Beds', 'price': '$4,900'}]","Pleasanton, CA 94566",https://www.apartments.com/1433-freeman-ln-ple...,3½ BAs,"3,100 Sq Ft","3,100",12 Month Lease,Available Now,NaN,"$4,900",NaN,NaN,NaN,NaN
98,Fairfield,Houses,CA,94534,"[{'bedroom_type': '3 Beds', 'price': '$2,899'}]","Fairfield, CA 94534",https://www.apartments.com/568-barbaresco-ct-f...,2½ BAs,"2,008 Sq Ft","2,008",,Available Now,NaN,"$2,899",NaN,NaN,NaN,NaN


In [32]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2020_Apartment.com_Houses_Data_page25-28.csv", index=False, header=True)